In [1]:
#Set your Imports
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Add the project root to sys.path
notebook_dir = os.path.dirname(os.path.abspath('data.ipynb'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from datasharing.datasharing import DataSharingClient

In [ ]:
duckdbpath="path/to/file/nameofyourduckdbfile.duckdb"
client = DataSharingClient(duckdb_path=duckdbpath)
#client = DataSharingClient(duckdb_path=duckdbpath,duckdb_region='us-east-1') Uncomment if cloest to us=east-1

In [ ]:
#s3_uri = "s3://oceandatachallengebucket/public/francetax.parquet" Uncomment this for us-east-1
s3_uri = "s3://oceandatachallenges/public/francetax.parquet"
view_name = "francetax"
client.create_view(s3_uri, view_name)

In [ ]:
query = "SELECT COUNT(*) FROM francetax;"
result_df = client.query(query)
print(result_df)

In [ ]:
# Confirm the new table has been created
tables = client.list_tables()
print(tables)

In [ ]:
# Define the query to create a new table from the view
query = "SELECT COUNT(*) FROM francetax;"
new_table_name = "francerowcount"

# Create a new table from the query
client.query(query, new_table_name)

In [ ]:
table_names = ["francerowcount"]
output_dir = "/home/christianocean/datasharing/data"

# Export the tables to CSV format
client.export_tables(table_names, output_dir, "csv")#parquet for parquet export